In [13]:
using GaussianMixtures, Distributions, LinearAlgebra, RData, Random

In [2]:
# Set a random seed to obtain the same results in all executions
Random.seed!(1);

In [13]:
# Load Gaussian distributions mixture data
mixture = load("data/ESL.mixture.rda", convert=true)

# Means of the bivariate Gaussian distributions:
# - From 1 to 10 the means belong to the BLUE class
# - And from position 11 to 20 the means belong to the ORANGE class
gaussian_means = mixture["ESL.mixture"]["means"];

20×2 Array{Float64,2}:
 -0.253433    1.74148
  0.266693    0.371234
  2.09647     1.23336
 -0.0612727  -0.208679
  2.70354     0.596828
  2.37721    -1.18641
  1.05691    -0.683894
  0.578884   -0.0683458
  0.624252    0.598738
  1.67335    -0.289316
  1.19937     0.248409
 -0.302561    0.945419
  0.0572723   2.41973
  1.32932     0.819226
 -0.0793842   1.6138
  3.50793     1.05299
  1.61392     0.671738
  1.00754     1.36831
 -0.454621    1.08607
 -1.79802     1.92978

In [4]:
# Currently, GaussianMixtures package doesn't allow to get the index (or the Gaussian distribution name) where the
# sample comes from.
# For this reason, I copied the code and adapted it: 
# https://github.com/davidavdav/GaussianMixtures.jl/blob/4c93c34d8b0627b3d8a3088f35a676aa1b356789/src/rand.jl#L48
function rand_with_gaussian_index(gmm::GMM, n::Int)
    x = Array{Float64}(undef, n, gmm.d)
    ## generate indices distriuted according to weights
    index::Vector{Int} = mapslices(findall, rand(Multinomial(1, gmm.w), n).!=0,
                                   dims=1)[:]
    
    gmmkind = kind(gmm)
    for i=1:gmm.n
        ind = findall(index.==i)
        nₓ = length(ind)
        if gmmkind == :diag
            x[ind,:] .= (vec(gmm.μ[i,:]) .+ vec(sqrt.(gmm.Σ[i,:])) .* randn(gmm.d, nₓ))' ## v0.5 arraymageddon
        elseif gmmkind == :full
            x[ind,:] .= rand(MvNormal(vec(gmm.μ[i,:]), forcesymmetric(covar(gmm.Σ[i]))), nₓ)'
        else
            error("Unknown kind")
        end
    end
    # Return the index too
    x, index
end

rand_with_gaussian_index (generic function with 1 method)

In [5]:
# Create a 20 bivariate Gaussians mixture
gaussians = 20
dimension = 2
gmm = GMM(gaussians, dimension) 

# Set means
gmm.μ = gaussian_means

# Set covariances
gmm.Σ = fill(1/5, (gaussians, dimension));

In [10]:
x_test, y_test = rand_with_gaussian_index(gmm, 10000)
# y_test contains the indexes of the Gaussian distributions: 
# if the index is higher than 10, then the class should be ORANGE (1), BLUE otherwise (0) 
y_test = y_test .> 10;

In [11]:
y_test

10000-element BitArray{1}:
 0
 0
 1
 1
 0
 0
 1
 0
 0
 0
 0
 0
 0
 ⋮
 0
 1
 0
 0
 1
 1
 1
 1
 1
 0
 0
 1

In [12]:
x_test

10000×2 Array{Float64,2}:
  1.80808     0.124657
  2.13182    -0.568647
  1.44236     0.35093
 -0.44959     2.65309
  1.88738    -1.19788
  0.876792    1.50304
 -0.256277    2.31647
  0.389238    0.559785
  0.0130878   0.0238688
  1.69821    -0.631582
 -0.420895    1.75088
  0.123937    0.600653
  0.0276842   0.129026
  ⋮          
  0.0745085  -0.336263
  0.0588848   2.02397
 -0.147954    2.35596
  0.670796   -0.438492
  1.26004     0.607064
  1.19855     0.082453
 -0.305228    1.03908
  3.57135     0.737773
 -1.17207     0.455179
  1.29536    -0.467395
  3.19122     1.21813
  0.904396   -0.0796924

In [2]:
# New try using data from mixture.rda

mixture = load("data/ESL.mixture.rda", convert=true)

Dict{String,Any} with 1 entry:
  "ESL.mixture" => DictoVec{Any}("x"=>[2.52609 0.32105; 0.366954 0.0314621; … ;…

In [3]:


marginal = mixture["ESL.mixture"]["marginal"]
prob = mixture["ESL.mixture"]["prob"];

In [30]:
# bayes.error<-sum(marginal*(prob*I(prob<0.5)+(1-prob)*I(prob>=.5)))

error = dot(marginal, prob .* Int.(prob .< 0.5) + (1 .- prob) .* Int.(prob .>= 0.5))

0.2101192033010554